In [1]:
import tensorflow as tf
import numpy as np

In [2]:
np.random.seed(777)

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
minist = input_data.read_data_sets('./dataset/minist/', one_hot=True)

Extracting ./dataset/minist/train-images-idx3-ubyte.gz
Extracting ./dataset/minist/train-labels-idx1-ubyte.gz
Extracting ./dataset/minist/t10k-images-idx3-ubyte.gz
Extracting ./dataset/minist/t10k-labels-idx1-ubyte.gz


In [4]:
print(minist.train.images.shape)
print(minist.train.labels.shape)
minist.train.num_examples

(55000, 784)
(55000, 10)


55000

In [5]:
print(minist.test.images.shape)
print(minist.test.labels.shape)
minist.test.num_examples

(10000, 784)
(10000, 10)


10000

In [6]:
batch_size = 100
len_epoch = 1000
rate = 0.1

In [7]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
KB = tf.placeholder(tf.float32)

In [8]:
# W1 = tf.Variable(tf.random_normal([784, 256]))
# W2 = tf.Variable(tf.random_normal([256, 256]))
# W3 = tf.Variable(tf.random_normal([256, 10]))

W1 = tf.get_variable('W1', [784, 256], initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable('W2', [256, 256], initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable('W3', [256, 10], initializer=tf.contrib.layers.xavier_initializer())

B1 = tf.Variable(tf.random_normal([256]))
B2 = tf.Variable(tf.random_normal([256]))
B3 = tf.Variable(tf.random_normal([10]))

L1 = tf.nn.relu(tf.matmul(X, W1)+B1)
L1 = tf.nn.dropout(L1, keep_prob=KB)

L2 = tf.nn.relu(tf.matmul(L1, W2)+B2)
L2 = tf.nn.dropout(L2, keep_prob=KB)

H = tf.matmul(L2, W3)+B3

In [9]:
C = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=H, labels=Y)
)

# H = tf.nn.softmax(tf.matmul(X, W) + b)
# C = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(H), axis=1))

In [10]:
T = tf.train.AdamOptimizer(rate).minimize(C)

In [11]:
P = tf.equal(
    (tf.arg_max(H, 1)), 
    (tf.arg_max(Y, 1))
)
A = tf.reduce_mean(tf.cast(P, tf.float32))

Instructions for updating:
Use `argmax` instead


In [12]:
# cost 94%
# cost 98%

In [13]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for epoch in range(15):
    avg_cost = 0

    total_batch = int(minist.train.num_examples / batch_size)

    for i in range(total_batch):
        trainx, trainy = minist.train.next_batch(100)
        _, c = sess.run([T, C], feed_dict={X: trainx, Y: trainy, KB: 0.7})

        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.3f}'.format(avg_cost))

a = sess.run(A, feed_dict={X: minist.test.images, Y: minist.test.labels, KB: 1})
print('{0} %'.format(a))

Epoch: 0001 cost = 2.687
Epoch: 0002 cost = 1.882
Epoch: 0003 cost = 1.948
Epoch: 0004 cost = 1.959
Epoch: 0005 cost = 2.042
Epoch: 0006 cost = 2.137
Epoch: 0007 cost = 2.149
Epoch: 0008 cost = 2.157
Epoch: 0009 cost = 2.168
Epoch: 0010 cost = 2.271
Epoch: 0011 cost = 2.281
Epoch: 0012 cost = 2.251
Epoch: 0013 cost = 2.198
Epoch: 0014 cost = 2.238
Epoch: 0015 cost = 2.232
0.1534000039100647 %


In [14]:
h = sess.run(H, feed_dict={X: [minist.test.images[0]] ,KB: 1})
# len(h)
h
np.argmax(h)
# minist.test.labels[0]

4